In [1]:
import os
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [6]:
try:
    connection = mysql.connector.connect(
        host='127.0.0.1',
        port=3307,
        database=os.getenv('MYSQL_DATABASES'),
        user=os.getenv('MYSQL_ROOT_USER'),
        password=os.getenv('MYSQL_ROOT_PASSWORD')
    )
except Error as e:
    print("Lỗi kết nối MySQL:", e)
    exit()

In [7]:
folder_path = '../data/'

In [8]:
file_path = os.path.join(folder_path, 'companies.csv')
df = pd.read_csv(file_path)
cursor = connection.cursor()
for _, row in df.iterrows():
    query1 = "INSERT INTO companies (symbol, comGroupCode, organName, organShortName, organTypeCode, icbName, sector, industry, `group`, subGroup, icbCode) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    value1 = (row['ticker'], row['comGroupCode'],
              row['organName'], row['organShortName'], row['organTypeCode'], row['icbName'],
              row['sector'], row['industry'], row['group'], row['subgroup'], row['icbCode'])
    cursor.execute(query1, value1)
        
connection.commit()
cursor.close()

True

In [9]:
file_path = os.path.join(folder_path, 'all_stocks.csv')
df = pd.read_csv(file_path)
df = df.drop(["Time"], axis=1)
df['TradingDate'] = pd.to_datetime(df['TradingDate'], dayfirst=True)
# df.head()
cursor = connection.cursor()
for _, row in df.iterrows():
    query2 = """
        INSERT INTO Trades (symbol, value, tradingDate, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    value2 = (row['Symbol'], row['Value'], row['TradingDate'], row['Open'], row['High'],
              row['Low'], row['Close'], row['Volume'])
    cursor.execute(query2, value2)
    
connection.commit()
cursor.close()

True

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.sql.expression import select, text

engine = create_engine('trino://trino@localhost:8085/iceberg')
connection = engine.connect()

rows = connection.execute(text("SELECT * FROM information_schema.tables")).fetchall()

In [ ]:
for i in rows:
    print(i)

In [2]:
import pandas as pd
import os, glob

In [3]:
path = "../data/"

In [ ]:
file_path = os.path.join(path, 'companies.csv')
file_path

In [ ]:
dff = pd.read_csv(file_path, index_col=None, header=0)
dff.head()

In [4]:
all_files = glob.glob(os.path.join(path, "*.csv"))
# all_files
l = []
for filename in all_files:
    if filename != "../data\\companies.csv":
        df = pd.read_csv(filename, index_col=None, header=0)
        l.append(df)

frame = pd.concat(l, ignore_index=True, axis=0)
# frame['TradingDate'] = pd.to_datetime(frame['TradingDate'], dayfirst=True)
# frame = frame.drop(["Unnamed: 0"], axis=1)
frame.head()

,Symbol,Value,TradingDate,Time,Open,High,Low,Close,Volume
0,ACB,4.397575e+10,02/01/2013,NaN,16300,17400,16300,17300,2578600
1,ACB,3.379012e+10,03/01/2013,NaN,17500,17800,16700,17000,1967200
2,ACB,1.989316e+10,04/01/2013,NaN,16900,17200,16700,17200,1170100
3,ACB,2.333590e+10,07/01/2013,NaN,17000,17500,17000,17200,1355700
4,ACB,3.749339e+10,08/01/2013,NaN,17100,17500,16900,17300,2180000


In [5]:
frame.to_csv('../data/all_stocks.csv', index=False)

In [50]:
import random


def generate_unique_sequence():
    while True:
        # Tạo dãy số ngẫu nhiên gồm 6 chữ số
        sequence = [random.randint(1, 9)] + random.sample(range(10), 5)

        # Kiểm tra tính duy nhất của dãy số
        if not 'previous_sequence' in locals() or sequence != previous_sequence:
            break

    # Lưu dãy số hiện tại để sử dụng làm dãy trước đó cho lần tiếp theo
    globals()['previous_sequence'] = sequence
    number_string = ''.join(map(str, sequence))
    result = int(number_string)
    return result


# Sử dụng hàm để tạo dãy số không trùng nhau
unique_sequence = generate_unique_sequence()
print(unique_sequence)

730798


In [118]:
import random

previous_number = []


def generate_unique_number():
    global previous_number
    while True:
        # Tạo dãy số ngẫu nhiên gồm 6 chữ số
        number = [random.randint(1, 9)] + random.sample(range(10), 5)

        # Kiểm tra tính duy nhất của dãy số
        if number != previous_number and number[0] != 0:
            break

    # Lưu dãy số hiện tại để sử dụng làm dãy trước đó cho lần tiếp theo
    previous_number = number
    number_string = ''.join(map(str, number))
    result = int(number_string)
    return result

print(generate_unique_number())

707138
